## The Trivia API and The Quizzler App
- [Trivia](https://opentdb.com/) API를 이용해서 지난시간 복습하기
- 해당 API를 통해서 Quizzler 앱 구축하기 

### 1. Trivia Question API
- Quizzler app을 만들기 위해 이전에 작업 했던 quiz quiz를 참조 
- 이번에는 Trivia API를 참조해서 데이터 불러오기 

In [ ]:
# quize_module/data.py
# Trivia Database API를 이용해서 데이터 불러오기 

# 모듈 불러오기 
import requests

# 파라미터 설정 
prameters = {
    "amount": 20,
    "type": "boolean"
}

response = requests.get("https://opentdb.com/api.php", params=prameters)   # 앤드포인트로 요청 받기 
response.raise_for_status()                                                # 응답코드 - 200이 아니면 예외를 발생 시킴 

question_data = response.json()["results"]                                  # Trivia API에서 가져온 데이터 저장
